# Libraries + Setting up PATH

In [1]:
import pandas as pd
from PIL import Image
from tqdm import tqdm
import os
from pathlib import Path
from glob import glob
import cv2

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.amp as amp
from torchsummary import summary

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import torchvision.transforms as transforms
import torchvision.transforms.v2 as v2

In [ ]:
from poseLandmark import poseLandmark

In [3]:
parentDir = Path(os.path.abspath(""))

# Dataset

## Dataset files

In [3]:
datasetDir = parentDir.joinpath("dataset-generator/data/poseLandmarkDataset")

In [4]:
csvFile = datasetDir.joinpath("coordinate.csv")
coordinateDf = pd.read_csv(csvFile)

In [5]:
coordinateDf

,image,head_x,head_y,neck_x,neck_y,torso_x,torso_y,left_shoulder_x,left_shoulder_y,left_elbow_x,...,left_foot_toe_x,left_foot_toe_y,right_hip_x,right_hip_y,right_knee_x,right_knee_y,right_heel_x,right_heel_y,right_foot_toe_x,right_foot_toe_y
0,20240925_192500_312.jpg,0.630556,0.205556,0.636111,0.258333,0.652778,0.402778,0.733333,0.305556,0.813889,...,0.708333,0.980556,0.597222,0.597222,0.586111,0.758333,0.580556,0.961111,0.530556,0.972222
1,20240928_105700_65.jpg,0.555556,0.044444,0.547222,0.127778,0.547222,0.288889,0.438889,0.169444,0.352778,...,0.450000,0.961111,0.608333,0.486111,0.613889,0.705556,0.630556,0.922222,0.677778,0.977778
2,20240925_192500_707.jpg,0.541667,0.233333,0.538889,0.313889,0.525000,0.450000,0.452778,0.352778,0.422222,...,0.441667,0.972222,0.569444,0.591667,0.613889,0.725000,0.644444,0.947222,0.691667,0.977778
3,20240928_105700_132.jpg,0.561111,0.072222,0.563889,0.147222,0.555556,0.302778,0.452778,0.188889,0.325000,...,0.458333,0.983333,0.619444,0.480556,0.613889,0.663889,0.625000,0.919444,0.663889,0.969444
4,20240928_105700_259.jpg,0.516667,0.044444,0.558333,0.086111,0.613889,0.288889,0.658333,0.197222,0.730556,...,0.616667,0.961111,0.550000,0.458333,0.522222,0.700000,0.572222,0.980556,0.488889,0.972222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,20240925_192500_487.jpg,0.491667,0.236111,0.497222,0.302778,0.488889,0.444444,0.419444,0.358333,0.422222,...,0.427778,0.994444,0.547222,0.608333,0.541667,0.758333,0.530556,0.966667,0.550000,0.994444
447,20240925_192500_32.jpg,0.522222,0.180556,0.516667,0.250000,0.508333,0.436111,0.422222,0.302778,0.363889,...,0.441667,0.988889,0.555556,0.622222,0.533333,0.822222,0.538889,0.991667,0.000000,0.000000
448,20240925_192500_690.jpg,0.647222,0.241667,0.638889,0.294444,0.650000,0.455556,0.558333,0.341667,0.516667,...,0.597222,0.980556,0.694444,0.597222,0.686111,0.750000,0.661111,0.947222,0.697222,0.972222
449,20240928_105700_696.jpg,0.000000,0.000000,0.000000,0.000000,0.550000,0.291667,0.219444,0.013889,0.266667,...,0.000000,0.000000,0.716667,0.613889,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Dataset stuff

In [6]:
class landmarksDataset(Dataset):
    def __init__(self, csvFile, imageDir, transforms=None):
        self.transforms = transforms
        self.imageDir = imageDir
        self.sample = []

        df = pd.read_csv(csvFile)
        imagesFile = df.pop("image").values
        landmarks = df
        for index in range(len(imagesFile)):
            landmarkCoordinates = landmarks.iloc[index].values
            self.sample.append((
                f"{imageDir}/{imagesFile[index]}",
                (
                    (landmarkCoordinates[0], landmarkCoordinates[1]),
                    (landmarkCoordinates[2], landmarkCoordinates[3]),
                    (landmarkCoordinates[4], landmarkCoordinates[5]),

                    (landmarkCoordinates[6], landmarkCoordinates[7]),
                    (landmarkCoordinates[8], landmarkCoordinates[9]),
                    (landmarkCoordinates[10], landmarkCoordinates[11]),
                    
                    (landmarkCoordinates[12], landmarkCoordinates[13]),
                    (landmarkCoordinates[14], landmarkCoordinates[15]),
                    (landmarkCoordinates[16], landmarkCoordinates[17]),
                    
                    (landmarkCoordinates[18], landmarkCoordinates[19]),
                    (landmarkCoordinates[20], landmarkCoordinates[21]),
                    (landmarkCoordinates[22], landmarkCoordinates[23]),
                    (landmarkCoordinates[24], landmarkCoordinates[25]),

                    (landmarkCoordinates[26], landmarkCoordinates[27]),
                    (landmarkCoordinates[28], landmarkCoordinates[29]),
                    (landmarkCoordinates[30], landmarkCoordinates[31]),
                    (landmarkCoordinates[32], landmarkCoordinates[33])
                )
                ))

    def __len__(self):
        return len(self.sample)
    
    def __getitem__(self, index):
        imgPath, landmarks = self.sample[index]
        landmarks = torch.tensor(landmarks)
        image = Image.open(imgPath).convert("RGB")
        if self.transforms:
            image = self.transforms(image) / 255
        return image, landmarks

In [7]:
def loadDataset(csvFile, imageDir, transforms=None, batchSize=16, splitRatio=0.2, worker=16):
    dataset = landmarksDataset(csvFile, imageDir, transforms)
    trainDataset, testDataset = train_test_split(dataset, test_size=splitRatio)
    print(f"Train dataset size: {len(trainDataset)}, Test dataset size: {len(testDataset)}")
    trainLoader = DataLoader(trainDataset, batch_size=batchSize, num_workers=worker)
    testLoader = DataLoader(testDataset, batch_size=batchSize, num_workers=worker)
    return trainLoader, testLoader

In [8]:
imageTransformers = transforms.Compose([
    v2.PILToTensor(),
    v2.ToDtype(torch.float32),
    v2.Resize(256)
])

## Dataset loading

In [9]:
trainLoader, testLoader = loadDataset(csvFile, datasetDir, imageTransformers)

Train dataset size: 360, Test dataset size: 91


# Model

## Model debugging

In [ ]:
debugModel = poseLandmark()
summary(debugModel, torch.zeros(16, 3, 256, 256))
torch.cuda.empty_cache()

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 65536]               --
|    └─Conv2d: 2-1                       [-1, 16, 256, 256]        448
|    └─ReLU: 2-2                         [-1, 16, 256, 256]        --
|    └─Conv2d: 2-3                       [-1, 64, 256, 256]        9,280
|    └─ReLU: 2-4                         [-1, 64, 256, 256]        --
|    └─MaxPool2d: 2-5                    [-1, 64, 128, 128]        --
|    └─BatchNorm2d: 2-6                  [-1, 64, 128, 128]        128
|    └─Conv2d: 2-7                       [-1, 128, 128, 128]       73,856
|    └─ReLU: 2-8                         [-1, 128, 128, 128]       --
|    └─Conv2d: 2-9                       [-1, 256, 128, 128]       295,168
|    └─ReLU: 2-10                        [-1, 256, 128, 128]       --
|    └─MaxPool2d: 2-11                   [-1, 256, 64, 64]         --
|    └─BatchNorm2d: 2-12                 [-1, 256, 64, 64]         512


In [7]:
debugModel = poseLandmark()
debugOutput = debugModel(torch.zeros(16, 3, 360, 360))
torch.cuda.empty_cache()

In [8]:
print(len(debugOutput))
print(debugOutput[0].shape)

17
torch.Size([16, 2])


## Model training

In [11]:
def train(model,
          epochs,
          scaler,
          lossFn,
          optimizer,
          trainLoader,
          validationLoader,
          checkpointPath,
          overfitDelta=0.005,
          patient=10,
          device="cuda"):
    history = {"trainAvgLoss" : [], "testAvgLoss" : []}
    model.to(device)
    bestLoss = 99999999999
    overfitStreak = 0
    try:
        for epoch in range(epochs + 1):
            # Training
            model.train()
            totalTrainLoss = 0
            totalTrainBatchCount = 0
            trainBar = tqdm(trainLoader, desc=f"Training epoch: {epoch} / {epochs}", unit="batch", leave=True)
            for batch in trainBar:
                loss = 0
                for index, landmark in enumerate(batch):
                    batch[index] = batch[index].to(device)
                image = batch[0]
                landmarks = batch[1:][0].permute(1, 0, 2)

                with torch.autocast(device_type=device, dtype=torch.bfloat16):
                    optimizer.zero_grad()
                    predictions = model(image)
                    for index, landmarkPrediction in enumerate(predictions):
                        loss += lossFn(landmarkPrediction, landmarks[index])

                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                totalTrainLoss += loss.item()
                totalTrainBatchCount += 1
            history["trainAvgLoss"].append(totalTrainLoss / totalTrainBatchCount)

            # Validation
            model.eval()
            totalValLoss = 0
            totalValBatchCount = 0
            validationBar = tqdm(validationLoader, desc=f"Validating epoch: {epoch} / {epochs}", unit="batch", leave=True)
            for batch in validationBar:
                loss = 0
                for index, landmark in enumerate(batch):
                    batch[index] = batch[index].to(device)
                image = batch[0]
                landmarks = batch[1:][0].permute(1, 0, 2)

                with torch.autocast(device_type=device, dtype=torch.bfloat16):
                    predictions = model(image)
                    for index, landmarkPrediction in enumerate(predictions):
                        loss += lossFn(landmarkPrediction, landmarks[index])

                totalValLoss += loss.item()
                totalValBatchCount += 1
            history["testAvgLoss"].append(totalValLoss / totalValBatchCount)

            # Early stopping
            if bestLoss > history["testAvgLoss"][-1] + overfitDelta:
                bestLoss = history["testAvgLoss"][-1]
                overfitStreak = 0
                torch.save(model.state_dict(), checkpointPath)
            else:
                overfitStreak += 1

            # Logging
            print(f"{'Average train loss: ':>30}{history['trainAvgLoss'][-1]:.5f}")
            print(f"{'Average test loss: ':>29}{history['testAvgLoss'][-1]:.5f}",end="\n\n")

            if overfitStreak >= patient:
                model.load_state_dict(torch.load(checkpointPath, weights_only=True))
                print("Model is going overfit")
                break
    except KeyboardInterrupt:
        model.load_state_dict(torch.load(checkpointPath, weights_only=True))
        print("Stop training due to keyboard interrupt")
    return history

In [12]:
torch.manual_seed(7800)
model = poseLandmark()
lossFn = nn.L1Loss()
optimizer = optim.AdamW(model.parameters(), lr=0.00001)
scaler = amp.GradScaler()
history = train(model=model,
                epochs=10000,
                scaler=scaler,
                lossFn=lossFn,
                optimizer=optimizer,
                trainLoader=trainLoader,
                validationLoader=testLoader,
                checkpointPath=parentDir.joinpath("model/poseLandmark.pth"),
                overfitDelta=0)

Validating epoch: 0 / 10000: 100%|██████████| 6/6 [00:00<00:00, 10.45batch/s]


          Average train loss: 3.02339
          Average test loss: 3.02457



Validating epoch: 1 / 10000: 100%|██████████| 6/6 [00:00<00:00, 10.92batch/s]


          Average train loss: 2.69794
          Average test loss: 2.47616



Validating epoch: 2 / 10000: 100%|██████████| 6/6 [00:00<00:00, 11.69batch/s]


...

          Average train loss: 1.17936
          Average test loss: 0.76530



Validating epoch: 164 / 10000: 100%|██████████| 6/6 [00:00<00:00,  9.84batch/s]


          Average train loss: 1.16968
          Average test loss: 0.74424



Validating epoch: 165 / 10000: 100%|██████████| 6/6 [00:00<00:00, 11.04batch/s]


          Average train loss: 1.17483
          Average test loss: 0.76742

Model is going overfit


In [ ]:
torch.cuda.empty_cache()